In [1]:
%pip install tensorflow tensorflow-hub opencv-python pandas matplotlib

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.0 MB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 1.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import math
import matplotlib.pyplot as plt

# Load MoveNet model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']


In [9]:

# Function to run pose estimation on a frame
def run_movenet(frame):
    # Resize and normalize image
    input_image = tf.convert_to_tensor(frame, dtype=tf.float32)
    input_image = tf.image.resize_with_pad(input_image, 192, 192)
    input_image = tf.expand_dims(input_image, axis=0)

    # Run model inference
    keypoints = movenet(input_image)['output_0'].numpy().reshape((17, 3))
    return keypoints

# Function to calculate angle between three points (hip, knee, ankle)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Angle calculation using cosine rule
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ac = np.linalg.norm(a - c)
    angle = math.degrees(math.acos((ab**2 + bc**2 - ac**2) / (2 * ab * bc)))
    return angle



In [10]:

# Open video
cap = cv2.VideoCapture('input_video.mp4')

landmarks_list = []
angles_list = []
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run pose estimation on each frame
    keypoints = run_movenet(frame)

    # Extract landmarks for relevant body parts (e.g., hip, knee, ankle)
    left_hip = keypoints[11][:2]
    left_knee = keypoints[13][:2]
    left_ankle = keypoints[15][:2]

    # Calculate the angle between upper and lower body
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    
    # Save keypoints and angles
    landmarks_list.append(keypoints.flatten().tolist())
    angles_list.append([frame_num, angle])

    frame_num += 1

cap.release()


In [11]:
# Convert the angle data to a DataFrame
angles_df = pd.DataFrame(angle_data)

# Convert 'frame' and 'angle' columns to numeric types (if necessary)
angles_df['frame'] = pd.to_numeric(angles_df['frame'], errors='coerce')
angles_df['angle'] = pd.to_numeric(angles_df['angle'], errors='coerce')

# Save the data to a CSV file
angles_df.to_csv('angles.csv', index=False)

# Plot the angle over time
angles_df.plot(x='frame', y='angle', kind='line', title='Angle between Upper and Lower Body over Time')
plt.xlabel('Frame')
plt.ylabel('Angle (degrees)')
plt.show()

NameError: name 'angle_data' is not defined